In [ ]:
import objaverse
import objaverse.xl as oxl
import pandas as pd
from typing import Any, Dict, Hashable
import tqdm
import urllib

import open3d as o3d

import pathlib

objaverse.__version__

In [ ]:
#----- Select all .obj files
annotations_df=oxl.get_annotations(download_dir="./output/.objaversexl_annotations")

selected_annotations_df=annotations_df[
    (annotations_df["fileType"]=="obj")]
print(f"Found {len(selected_annotations_df)} .obj files")

selected_annotations_df.head(50).to_csv("./output/objaversexl_obj_files.csv", index=False)

In [ ]:
#----- Download all .obj files with less than 500 faces
maxNumberOfFaces=500
download_dir=f"./output/objaversexl_obj_files_{maxNumberOfFaces}"
pathlib.Path(download_dir).mkdir(parents=True,exist_ok=True)

validObjects=[]


In [ ]:

def handle_found_object(local_path: str,
                        file_identifier: str,
                        sha256: str,
                        metadata: Dict[Hashable, Any]) -> None:
    
    #Open local mesh
    mesh=o3d.io.read_triangle_mesh(local_path)
    if len(mesh.triangles)<=maxNumberOfFaces :
        pass
        # validObjects.append({
        #     "local_path": local_path,
        #     "file_identifier": file_identifier,
        #     "sha256": sha256,
        #     "num_faces": len(mesh.triangles),
        #     **metadata
        # })
    else :
        pathlib.Path(local_path).unlink()

#oxl.download_objects(objects=selected_annotations_df, handle_found_object=handle_found_object, download_dir=f"./output/objaverse_obj_files_{maxNumberOfFaces}")

In [ ]:
for rowIndex in tqdm.tqdm(range(len(selected_annotations_df))):
    try :
        local_path=oxl.download_objects(selected_annotations_df.iloc[[rowIndex]], download_dir=download_dir, processes=1)
    except Exception as e:
        print(f"Error downloading object at index {rowIndex}: {e}")
        continue

    # urlFilePath=str(row.fileIdentifier)
    
    # if not row.source :
    #     row.source="unknown_source"
    # pathlib.Path(download_dir+"/"+row.source).mkdir(parents=True, exist_ok=True)
    # basefileName=pathlib.Path(urlFilePath).name
    # local_path=download_dir+"/"+row.source+"/"+basefileName
    # if not pathlib.Path(local_path).exists():
    #     try :
    #         cleaned_url=urllib.parse.quote(urlFilePath, safe=':/')
    #         urllib.request.urlretrieve(cleaned_url, local_path)
    #     except Exception as e:
    #         print(f"Error downloading {urlFilePath}: {e}")
    #         break

    #Open local mesh
    mesh=o3d.io.read_triangle_mesh(local_path)
    
    if len(mesh.triangles)>0 and len(mesh.triangles)<=maxNumberOfFaces :
        #pass
        validObjects.append({
                "local_path": local_path,
                "file_identifier": row.fileIdentifier,
                "sha256": row.sha256,
                "num_faces": len(mesh.triangles),
                **row.metadata
            })
    else :
        pathlib.Path(local_path).unlink()


print(f"Found {len(validObjects)} .obj files with less than {maxNumberOfFaces} faces.")
validObjects_df=pd.DataFrame(validObjects)
validObjects_df.to_csv(f"./output/objaverse_obj_files_less_than_{maxNumberOfFaces}_faces.csv", index=False)
